In [102]:
import numpy as np
import h5py
import torch
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

# Helper Clases / Functions

In [103]:
def checkAccuracy(model, dataloader):
    y_pred_tot = []
    y_sample_tot = []
    for i_batch, sample_batched in enumerate(dataloader):
        X_sample, y_sample = sample_batched
        # Flip axis first
        X_sample = torch.Tensor(np.moveaxis(X_sample.numpy(), -1, 0))
        X_sample, y_sample = Variable(X_sample), Variable(y_sample)
        scores = model(X_sample)
        #loss = loss_fn(scores, y_sample.long())
        #print('val loss: %f' % loss)
        _, y_pred = torch.max(scores, 1)
        y_pred_tot.append(y_pred.data.numpy())
        y_sample_tot.append(y_sample.data.numpy())
    acc = np.mean(np.concatenate(y_pred_tot) == np.concatenate(y_sample_tot))
    return acc
class Flatten(nn.Module):
    def forward(self, x):
        out, _ = x
        T, N, H = out.size() 
        return out.view(H, -1)

# Load Data

In [104]:
A01T = h5py.File('./../project_datasets/A01T_slice.mat', 'r')
print (A01T)
X = np.copy(A01T['image'])
y = np.copy(A01T['type'])
X = X[:, 0:23, :] # only goes to 23 since you dont use last 3 channels
# X = np.moveaxis(X, -1, 0)
y = y[0,0:X.shape[0]:1]
#subtract 769 so classes start at 0
y = y - 769
N, E, T = np.shape(X)
print (np.shape(X))

<HDF5 file "A01T_slice.mat" (mode r)>
(288, 23, 1000)


# Make DataLoaders

In [105]:
bs = 50
data = data_utils.TensorDataset(torch.Tensor(X), torch.Tensor(y))
dset = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [N-100, 50, 50])
dataloaders = {x: data_utils.DataLoader(dset[x], batch_size=bs, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}

# Define Model

In [106]:
class myLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, num_class):
        super(myLSTM, self).__init__()
        self.num_layer = num_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layer)
                            # batch_first=True)
        self.classifier = nn.Linear(hidden_dim, num_class)
    def forward(self, x):
        h0 = Variable(torch.zeros(self.num_layer, x.size(1), self.hidden_dim)).cuda()
        c0 = Variable(torch.zeros(self.num_layer, x.size(1), self.hidden_dim)).cuda()
        out, _ = self.lstm(x)
        out = out[-1, :, :]
        out = self.classifier(out)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            # Flip axis first
            X_sample = torch.Tensor(np.moveaxis(X_sample.numpy(), -1, 0))
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            out = self.forward(X_sample.cuda())
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.cpu().numpy() == y_sample.data.cpu().numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label

# Test the Model

In [109]:
dtype = torch.cuda.FloatTensor
hidden_dim = 50
num_classes = 4
num_epoches = 10
model = myLSTM(E, hidden_dim, 10, num_classes)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(num_epoches):
    for i, data in enumerate(dataloaders['train'], 0):
        X_train, y_train = data
        # Wrap them in Variable
        # Flip axis first
        X_train = torch.Tensor(np.moveaxis(X_train.numpy(), -1, 0))
        X_train, y_train = Variable(X_train), Variable(y_train)
        # forward + backward + optimize
        out = model(X_train.cuda())
        # print (out)
        loss = loss_fn(out, y_train.long().cuda())
        print('(%d batch) loss: %f' % (i, loss))
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_acc = model.check_accuracy(dataloaders['train'])
    val_acc = model.check_accuracy(dataloaders['val'])
    print('(Epoch %d / %d) train_acc: %f; val_acc: %f' % (epoch+1, num_epoches, train_acc, val_acc))
        

(0 batch) loss: 1.390534
(1 batch) loss: 1.396682
(2 batch) loss: 1.382214
(3 batch) loss: 1.392223
(Epoch 1 / 10) train_acc: 0.250000; val_acc: 0.220000
(0 batch) loss: 1.388496
(1 batch) loss: 1.389741
(2 batch) loss: 1.387006
(3 batch) loss: 1.386411
(Epoch 2 / 10) train_acc: 0.250000; val_acc: 0.220000
(0 batch) loss: 1.390685
(1 batch) loss: 1.390794
(2 batch) loss: 1.373139
(3 batch) loss: 1.394947


Process Process-2442:
Process Process-2443:
Process Process-2444:
Process Process-2441:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiproces

KeyboardInterrupt: 